In [2]:
import pandas as pd

In [18]:
df = pd.read_csv("data/Sales.csv")

In [19]:
df.shape

(9800, 18)

In [22]:
df.describe()

,Row ID,Postal Code,Sales
count,9800.000000,9789.000000,9800.000000
mean,4900.500000,55273.322403,230.769059
std,2829.160653,32041.223413,626.651875
min,1.000000,1040.000000,0.444000
25%,2450.750000,23223.000000,17.248000
50%,4900.500000,58103.000000,54.490000
75%,7350.250000,90008.000000,210.605000
max,9800.000000,99301.000000,22638.480000


In [24]:
df['Sales'].sum() / df['Order ID'].nunique()

np.float64(459.4751691791955)

In [26]:
df['Order Date'].unique()

array(['08/11/2017', '12/06/2017', '11/10/2016', ..., '18/06/2015',
       '28/02/2018', '09/05/2016'], shape=(1230,), dtype=object)

In [7]:
df = df.head()

In [8]:
date_dimension = pd.DataFrame({
    "Full_Date": pd.to_datetime(df["Order Date"]),
    "Year": pd.to_datetime(df["Order Date"]).dt.year,
    "Quarter": pd.to_datetime(df["Order Date"]).dt.quarter,
    "Month": pd.to_datetime(df["Order Date"]).dt.month,
    "Week": pd.to_datetime(df["Order Date"]).dt.isocalendar().week,
    "Day_Of_Week": pd.to_datetime(df["Order Date"]).dt.day_name(),
}).drop_duplicates().reset_index(drop=True)

# Customer Dimension Table
customer_dimension = df[["Customer ID", "Customer Name", "Segment"]].drop_duplicates().reset_index(drop=True)
customer_dimension.rename(columns={
    "Customer ID": "Customer_ID",
    "Customer Name": "Customer_Name"
}, inplace=True)

# Product Dimension Table
product_dimension = df[["Product ID", "Product Name", "Category", "Sub-Category"]].drop_duplicates().reset_index(drop=True)
product_dimension.rename(columns={
    "Product ID": "Product_ID",
    "Product Name": "Product_Name",
    "Sub-Category": "Sub_Category"
}, inplace=True)

# Location Dimension Table
location_dimension = df[["Country", "Region", "State", "City", "Postal Code"]].drop_duplicates().reset_index(drop=True)
location_dimension.rename(columns={"Postal Code": "Postal_Code"}, inplace=True)

# Ship Mode Dimension Table
ship_mode_dimension = df[["Ship Mode"]].drop_duplicates().reset_index(drop=True)
ship_mode_dimension.rename(columns={"Ship Mode": "Ship_Mode"}, inplace=True)

# Sales Fact Table
sales_fact = pd.DataFrame({
    "Order_ID": df["Order ID"],
    "Date_Key": pd.factorize(df["Order Date"])[0] + 1,
    "Ship_Date_Key": pd.factorize(df["Ship Date"])[0] + 1,
    "Customer_Key": pd.factorize(df["Customer ID"])[0] + 1,
    "Product_Key": pd.factorize(df["Product ID"])[0] + 1,
    "Location_Key": pd.factorize(df["City"] + df["State"] + df["Postal Code"].astype(str))[0] + 1,
    "Ship_Mode_Key": pd.factorize(df["Ship Mode"])[0] + 1,
    "Sales": df["Sales"],
})

In [9]:

sales_fact

,Order_ID,Date_Key,Ship_Date_Key,Customer_Key,Product_Key,Location_Key,Ship_Mode_Key,Sales
0,CA-2017-152156,1,1,1,1,1,1,261.9600
1,CA-2017-152156,1,1,1,2,1,1,731.9400
2,CA-2017-138688,2,2,2,3,2,1,14.6200
3,US-2016-108966,3,3,3,4,3,2,957.5775
4,US-2016-108966,3,3,3,5,3,2,22.3680


In [10]:
product_dimension

,Product_ID,Product_Name,Category,Sub_Category
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
